In [1]:
%pip uninstall -y transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to c:\users\abhi4\appdata\local\temp\pip-req-build-zc69x6mp
  Resolved https://github.com/huggingface/transformers to commit fce52cefa744a5900fc065aafb2f55d846d1202c
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/269.5 kB ? eta -:--:--
     ---- ---------------------------------- 30.7/269.5 kB 1.4 MB/s eta 0:00:01
     --------- --------------------------- 71.7/269.5 kB 787.7 kB/s eta 0:00:01
     ------------------------------- ------ 225.3/269.5 kB 1.5 MB/s eta 0:00:01
     ------------------------------------ - 256.0/269.5 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 269.5/269.5 k

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\abhi4\AppData\Local\Temp\pip-req-build-zc69x6mp'

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: c:\Users\abhi4\src\GitHub\Wikisum\venv\Scripts\python.exe -m pip install --upgrade pip


In [3]:
%pip install huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: c:\Users\abhi4\src\GitHub\Wikisum\venv\Scripts\python.exe -m pip install --upgrade pip


In [4]:
%pip install torch torchtext sentencepiece pandas tqdm datasets peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7 accelerate einops scipy

     ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/198.6 MB 1.4 MB/s eta 0:02:26
     ---------------------------------------- 0.1/198.6 MB 1.1 MB/s eta 0:03:09
     ---------------------------------------- 0.2/198.6 MB 1.7 MB/s eta 0:01:56
     ---------------------------------------- 0.4/198.6 MB 2.4 MB/s eta 0:01:22
     ---------------------------------------- 0.8/198.6 MB 3.4 MB/s eta 0:00:58
     ---------------------------------------- 1.3/198.6 MB 4.8 MB/s eta 0:00:42
     ---------------------------------------- 2.1/198.6 MB 6.6 MB/s eta 0:00:30
      -------------------------------------- 4.2/198.6 MB 11.6 MB/s eta 0:00:17
      -------------------------------------- 4.3/198.6 MB 11.4 MB/s eta 0:00:18
     - ------------------------------------- 6.0/198.6 MB 13.3 MB/s eta 0:00:15
     - ------------------------------------- 6.9/198.6 MB 14.6 MB/s eta 0:00:14
     - ------------------------------------- 6.


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: c:\Users\abhi4\src\GitHub\Wikisum\venv\Scripts\python.exe -m pip install --upgrade pip


# Prepare the Data Set for Fine tuning

In [5]:
import os
from dataclasses import dataclass, field
from typing import Optional

from datasets import load_dataset
from datasets import load_from_disk
from datasets import Dataset

c:\Users\abhi4\src\GitHub\Wikisum\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
wikisum_data = load_dataset("abertsch/rescraped-wikisum-test")

Generating train split: 100%|██████████| 180889/180889 [00:02<00:00, 70089.60 examples/s]


In [7]:
import pandas as pd
# Convert to DataFrame
df = pd.DataFrame(wikisum_data)

# Display the first few rows of the DataFrame
df.head(2)

,train
0,"{'id': '0.1', 'input': 'The Excellent Dizzy Co..."
1,"{'id': '0.2', 'input': 'A break away!<EOT>Not ..."


In [27]:
df.iloc[0, 0]['input']

'The Excellent Dizzy Collection<EOT>Early on I was pretty useless at the game, getting stuck on the screen with the fire breathing dragon and the bull in the cave in particular. Luckily some help and discussion was at hand when I went along with my dad to his pigeon club meetings every week. One of the other pigeon fanciers brought their grandson along, who happened to play the Dizzy games a lot on his Amstrad CPC . He was a few years older, and I managed to get valuable tips and hints off him each time when I got to a particular screen (Such as giving the meaty bone to the bull to get him to sit in the cave). I used to get very excited about finding out how to get past certain parts, in the days before there was the internet to look up the full solutions and this poor lad used to get interrogated each week as I progressed through the game, eager to find out what was on the next screen.\n1'

In [28]:
df.iloc[0, 0]['title']

'The Excellent Dizzy Collection'

In [29]:
df.iloc[0, 0]['summary']

'The Excellent Dizzy Collection is a video game compilation published by Codemasters in November 1993. The title includes three stand alone games, based on the video game character Dizzy created by the Oliver Twins. The compilation contains, Dizzy the Adventurer, Panic Dizzy and the previously unreleased Go! Dizzy Go!. '

In [30]:
df.iloc[0, 0]['full_article']

"The Excellent Dizzy Collection is a video game compilation published by Codemasters in November 1993. The title includes three stand alone games, based on the video game character Dizzy created by the Oliver Twins. The compilation contains, Dizzy the Adventurer, Panic Dizzy and the previously unreleased Go! Dizzy Go!. == Background == After the failure of the Aladdin Deck Enhancer, Camerica was forced to close down, and Codemasters was in financial trouble. As a result, many staff left the Oliver twins' software company, Interactive Studios, who were unable to find replacements. Dizzy The Adventurer was freely bundled with the Aladdin, and three other titles were being developed for it; Dreamworld Pogie, Go! Dizzy Go! and Wonderland Dizzy. However, after the Aladdin was abandoned, the Oliver twins decided to release all four (when completed) on the Sega Master System and Sega Game Gear to bring in some extra revenue. However, Codemasters wasn’t comfortable releasing each as a full pri

In [24]:
df.loc[0]['train'].keys()

dict_keys(['id', 'input', 'title', 'summary', 'full_article', 'section_boundaries'])

# Transform each row into (summary, Article)

In [31]:
# Function to transform the row into desired format
def format_row(row):
    summary = row['train']['input']
    full_article = row['train']['full_article']
    title = row['train']['title']
    info = row['train']['input']

    userContent = f"""You are a helpful writing assistant. Your job is to write a wikipedia style article given the title, summary and information about the topic
    
    title: {title}

    summary: {summary}

    information: {info}
    """
    assistantContent = f"{full_article}"

    formatted_string = f"<s>[INST] {userContent} [/INST] {full_article}</s>"
    return formatted_string

# Apply the function to each row of the dataframe
df['Formatted'] = df.apply(format_row, axis=1)

# Display the formatted column
df['Formatted']

0         <s>[INST] You are a helpful writing assistant....
1         <s>[INST] You are a helpful writing assistant....
2         <s>[INST] You are a helpful writing assistant....
3         <s>[INST] You are a helpful writing assistant....
4         <s>[INST] You are a helpful writing assistant....
                                ...                        
180884    <s>[INST] You are a helpful writing assistant....
180885    <s>[INST] You are a helpful writing assistant....
180886    <s>[INST] You are a helpful writing assistant....
180887    <s>[INST] You are a helpful writing assistant....
180888    <s>[INST] You are a helpful writing assistant....
Name: Formatted, Length: 180889, dtype: object

In [32]:
new_df = df.rename(columns={'Formatted': 'Text'})
new_df = new_df[['Text']]

In [33]:
new_df.head()

,Text
0,<s>[INST] You are a helpful writing assistant....
1,<s>[INST] You are a helpful writing assistant....
2,<s>[INST] You are a helpful writing assistant....
3,<s>[INST] You are a helpful writing assistant....
4,<s>[INST] You are a helpful writing assistant....


In [34]:
# training_dataset = load_dataset("csv", data_files="formatted_data.csv", split="train")
from datasets import Dataset
training_dataset = Dataset.from_pandas(new_df)

In [42]:
%pip install -U ipywidgets "huggingface_hub[cli]"

     ---------------------------------------- 0.0/67.7 kB ? eta -:--:--
     ------------------ --------------------- 30.7/67.7 kB 1.4 MB/s eta 0:00:01
     ------------------ --------------------- 30.7/67.7 kB 1.4 MB/s eta 0:00:01
     ------------------ --------------------- 30.7/67.7 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 67.7/67.7 kB 410.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: c:\Users\abhi4\src\GitHub\Wikisum\venv\Scripts\python.exe -m pip install --upgrade pip


In [45]:
training_dataset.push_to_hub("AbhishekJosyula/Wikisun_QA_Format")

Uploading the dataset shards: 100%|██████████| 3/3 [03:46<00:00, 75.53s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/AbhishekJosyula/Wikisun_QA_Format/commit/45f6b6fc3e9e97593551c9841ad6af4305b25eac', commit_message='Upload dataset', commit_description='', oid='45f6b6fc3e9e97593551c9841ad6af4305b25eac', pr_url=None, pr_revision=None, pr_num=None)